In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

from neuralforecast import NeuralForecast
from neuralforecast.models import LSTM
from neuralforecast.losses.pytorch import MAE
import torch.optim as optim

from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error

In [ ]:
# Custom Metrics
def mean_bias_error(y_true, y_pred):
    return np.mean(y_pred - y_true)


def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    epsilon = 1e-10
    return np.mean(
        2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + epsilon)
    )


def rmae(y_true, y_pred, naive_forecast):
    return mean_absolute_error(y_true, y_pred) / mean_absolute_error(
        y_true, naive_forecast
    )


# Set plot style
sns.set_theme(style="whitegrid")

# States and Weeks
states = ["VIC", "SA", "NSW", "QLD", "TAS"]
forecast_windows = {
    1: ("2025-03-01 00:00:00", "2025-03-07 23:00:00"),
    2: ("2025-03-08 00:00:00", "2025-03-14 23:00:00"),
    3: ("2025-03-15 00:00:00", "2025-03-21 23:00:00"),
    4: ("2025-03-22 00:00:00", "2025-03-28 23:00:00"),
}
weeks = {1: "Week 1", 2: "Week 2", 3: "Week 3", 4: "Week 4"}

for state in states:
    print(f"Processing state: {state}")
    # Load and preprocess data
    df = pd.read_csv(
        f"../data/featurised/{state}_test_final.csv", parse_dates=["SETTLEMENTDATE"]
    )
    df.rename(columns={"SETTLEMENTDATE": "ds", "asinh_RRP": "y"}, inplace=True)
    df["unique_id"] = state
    rrp_median = df["rrp_median"].values[0]
    rrp_mad = df["rrp_mad"].values[0]

    exog_cols = [
        "asinh_RRP_lag_168",
        "TOTALDEMAND",
        "NETINTERCHANGE",
        "SS_SOLAR_UIGF",
        "SS_WIND_UIGF",
        "tempC",
    ]
    req_cols = ["unique_id", "ds", "y"]
    df = df[req_cols + exog_cols]
    df.sort_values(by=["unique_id", "ds"], inplace=True)
    df.set_index("ds", inplace=True)
    df = df.asfreq("h").reset_index()

    # Prepare combined results and metrics storage
    all_results_df = pd.DataFrame()
    metrics_df = pd.DataFrame()

    for week in range(1, 5):
        print(f"Processing week: {weeks[week]}")
        forecast_start, forecast_end = forecast_windows[week]

        Y_train_df = df[df.ds < forecast_start].copy()
        Y_test_df = df[(df.ds >= forecast_start) & (df.ds <= forecast_end)].copy()

        # Define LSTM model
        nf = NeuralForecast(
            models=[
                LSTM(
                    h=168,
                    input_size=336,
                    loss=MAE(),
                    scaler_type="minmax",
                    encoder_n_layers=3,
                    encoder_hidden_size=64,
                    decoder_hidden_size=64,
                    decoder_layers=3,
                    max_steps=250,
                    futr_exog_list=exog_cols,
                    early_stop_patience_steps=50,
                    recurrent=False,
                    random_seed=42,
                ),
            ],
            freq="h",
        )

        # Train the model
        val_len = 24 * 7 * 4
        nf.fit(df=Y_train_df, val_size=val_len)

        # Make predictions
        forecasts = nf.predict(futr_df=Y_test_df)
        Y_test_df = Y_test_df.reset_index(drop=True)
        forecasts = forecasts.reset_index(drop=True)

        # Merge predictions with actual values
        results_df = pd.merge(
            Y_test_df[["ds", "y"]], forecasts[["ds", "LSTM"]], on="ds", how="left"
        )

        results_df["y"] = np.sinh(results_df["y"]) * rrp_mad + rrp_median
        results_df["LSTM"] = np.sinh(results_df["LSTM"]) * rrp_mad + rrp_median

        # Naive Forecast (last 168 hours of training)
        Y_train_df["y"] = np.sinh(Y_train_df["y"]) * rrp_mad + rrp_median
        naive_forecast = Y_train_df["y"].iloc[-168:].reset_index(drop=True)
        results_df["Naive"] = naive_forecast.values

        # Add week identifier
        results_df["week"] = weeks[week]

        # Append to combined results for the state
        all_results_df = pd.concat([all_results_df, results_df], ignore_index=True)

        # Calculate metrics for LSTM and Naive
        for model in ["LSTM", "Naive"]:
            metrics = {
                "MAE": float(mean_absolute_error(results_df["y"], results_df[model])),
                "rMAE": float(
                    rmae(results_df["y"], results_df[model], results_df["Naive"])
                ),
                "RMSE": float(
                    root_mean_squared_error(results_df["y"], results_df[model])
                ),
                "MBE": float(mean_bias_error(results_df["y"], results_df[model])),
                "SMAPE": float(
                    symmetric_mean_absolute_percentage_error(
                        results_df["y"], results_df[model]
                    )
                ),
                "R2": float(r2_score(results_df["y"], results_df[model])),
            }

            print(f"Metrics for {model}: {metrics}")

            # Add to metrics_df
            week_metrics_df = pd.DataFrame(
                metrics, index=[f"{model} ({weeks[week]})"]
            ).T
            metrics_df = pd.concat([metrics_df, week_metrics_df], axis=1)

        print(f"Completed: {state} - {weeks[week]}")

    # Save all metrics for this state
    metrics_df.to_csv(
        f"./asinh_lstm_outputs/{state}/{state}_metrics_all_weeks.csv", index=True
    )

    # Save all results for this state
    all_results_df.to_csv(
        f"./asinh_lstm_outputs/{state}/{state}_results_all_weeks.csv", index=False
    )

    print(f"Saved consolidated metrics and results for {state}")

print("All states and weeks processed successfully.")

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

from neuralforecast import NeuralForecast
from neuralforecast.models import KAN
from neuralforecast.losses.pytorch import MAE

from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error

In [2]:
# Custom Metrics
def mean_bias_error(y_true, y_pred):
    return np.mean(y_pred - y_true)


def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    epsilon = 1e-10
    return np.mean(
        2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + epsilon)
    )


def rmae(y_true, y_pred, naive_forecast):
    return mean_absolute_error(y_true, y_pred) / mean_absolute_error(
        y_true, naive_forecast
    )


# Set plot style
sns.set_theme(style="whitegrid")

# States and Weeks
states = ["VIC", "SA", "NSW", "QLD", "TAS"]
forecast_windows = {
    1: ("2025-03-01 00:00:00", "2025-03-07 23:00:00"),
    2: ("2025-03-08 00:00:00", "2025-03-14 23:00:00"),
    3: ("2025-03-15 00:00:00", "2025-03-21 23:00:00"),
    4: ("2025-03-22 00:00:00", "2025-03-28 23:00:00"),
}
weeks = {1: "Week 1", 2: "Week 2", 3: "Week 3", 4: "Week 4"}

for state in states:
    print(f"Processing state: {state}")
    # Load and preprocess data
    df = pd.read_csv(
        f"../data/featurised/{state}_test_final.csv", parse_dates=["SETTLEMENTDATE"]
    )
    df.rename(columns={"SETTLEMENTDATE": "ds", "asinh_RRP": "y"}, inplace=True)
    df["unique_id"] = state
    rrp_median = df["rrp_median"].values[0]
    rrp_mad = df["rrp_mad"].values[0]

    exog_cols = [
        "asinh_RRP_lag_168",
        "TOTALDEMAND",
        "NETINTERCHANGE",
        "SS_SOLAR_UIGF",
        "SS_WIND_UIGF",
        "tempC",
    ]
    req_cols = ["unique_id", "ds", "y"]
    df = df[req_cols + exog_cols]
    df.sort_values(by=["unique_id", "ds"], inplace=True)
    df.set_index("ds", inplace=True)
    df = df.asfreq("h").reset_index()

    # Prepare combined results and metrics storage
    all_results_df = pd.DataFrame()
    metrics_df = pd.DataFrame()

    for week in range(1, 5):
        print(f"Processing week: {weeks[week]}")
        forecast_start, forecast_end = forecast_windows[week]

        Y_train_df = df[df.ds < forecast_start].copy()
        Y_test_df = df[(df.ds >= forecast_start) & (df.ds <= forecast_end)].copy()

        # Define KAN model
        nf = NeuralForecast(
            models=[
                KAN(
                    h=168,
                    input_size=336,
                    loss=MAE(),
                    max_steps=250,
                    n_hidden_layers=1,
                    grid_size=5,
                    spline_order=4,
                    scaler_type="minmax",
                    futr_exog_list=exog_cols,
                    early_stop_patience_steps=50,
                    random_seed=42,
                ),
            ],
            freq="h",
        )

        # Train the model
        val_len = 24 * 7 * 4
        nf.fit(df=Y_train_df, val_size=val_len)

        # Make predictions
        forecasts = nf.predict(futr_df=Y_test_df)
        Y_test_df = Y_test_df.reset_index(drop=True)
        forecasts = forecasts.reset_index(drop=True)

        # Merge predictions with actual values
        results_df = pd.merge(
            Y_test_df[["ds", "y"]], forecasts[["ds", "KAN"]], on="ds", how="left"
        )
        results_df["y"] = np.sinh(results_df["y"]) * rrp_mad + rrp_median
        results_df["KAN"] = np.sinh(results_df["KAN"]) * rrp_mad + rrp_median

        # Naive Forecast (last 168 hours of training)
        Y_train_df["y"] = np.sinh(Y_train_df["y"]) * rrp_mad + rrp_median
        naive_forecast = Y_train_df["y"].iloc[-168:].reset_index(drop=True)
        results_df["Naive"] = naive_forecast.values

        # Add week identifier
        results_df["week"] = weeks[week]

        # Append to combined results for the state
        all_results_df = pd.concat([all_results_df, results_df], ignore_index=True)

        # Calculate metrics for KAN and Naive
        for model in ["KAN", "Naive"]:
            metrics = {
                "MAE": float(mean_absolute_error(results_df["y"], results_df[model])),
                "rMAE": float(
                    rmae(results_df["y"], results_df[model], results_df["Naive"])
                ),
                "RMSE": float(
                    root_mean_squared_error(results_df["y"], results_df[model])
                ),
                "MBE": float(mean_bias_error(results_df["y"], results_df[model])),
                "SMAPE": float(
                    symmetric_mean_absolute_percentage_error(
                        results_df["y"], results_df[model]
                    )
                ),
                "R2": float(r2_score(results_df["y"], results_df[model])),
            }

            print(f"Metrics for {model}: {metrics}")

            # Add to metrics_df
            week_metrics_df = pd.DataFrame(
                metrics, index=[f"{model} ({weeks[week]})"]
            ).T
            metrics_df = pd.concat([metrics_df, week_metrics_df], axis=1)

        print(f"Completed: {state} - {weeks[week]}")

    # Save all metrics for this state
    metrics_df.to_csv(
        f"./asinh_kan_outputs/{state}/{state}_metrics_all_weeks.csv", index=True
    )

    # Save all results for this state
    all_results_df.to_csv(
        f"./asinh_kan_outputs/{state}/{state}_results_all_weeks.csv", index=False
    )

    print(f"Saved consolidated metrics and results for {state}")

print("All states and weeks processed successfully.")

Seed set to 42


Processing state: VIC
Processing week: Week 1


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | layers       | ModuleList    | 19.9 M | train
-------------------------------------------------------
19.9 M    Trainable params
0         Non-trainable params
19.9 M    Total params
79.479    Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=250` reached.
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

Metrics for KAN: {'MAE': 54.383011971731015, 'rMAE': 1.7060957425785532, 'RMSE': 61.827160300800486, 'MBE': 53.164658815293066, 'SMAPE': 0.4713675846427679, 'R2': -2.0873344514085286}
Metrics for Naive: {'MAE': 31.87570932539683, 'rMAE': 1.0, 'RMSE': 43.94874718695716, 'MBE': -4.4005406746031746, 'SMAPE': 0.42444227365299303, 'R2': -0.5599765797781004}
Completed: TAS - Week 4
Saved consolidated metrics and results for TAS
All states and weeks processed successfully.
